In [46]:
import ast
import re
import nltk
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pickle

<h1>Import Dataset</h1>

In [47]:
base = "data/"
has_face_pkl_path = "data/has_face.pkl"
reciprocity_path = "data/reciprocity.pkl"

# Get tweets corpus and join to dataset (may skip running if dataset is updated)

In [48]:
df_users = pd.read_csv("data\combined_twitter_data.csv")

In [49]:
# get tweets df
filenames_tweets = [
    "data/all tweets 2017/tweets_fake_followers.csv",
    "data/all tweets 2017/tweets_genuine_accounts.csv",
    "data/all tweets 2017/tweets_social_spambots_1.csv",
    "data/all tweets 2017/tweets_social_spambots_2.csv",
    "data/all tweets 2017/tweets_social_spambots_3.csv",
    "data/all tweets 2017/tweets_traditional_spambots_1.csv",

    "data/tweets 2015/tweets_E13.csv",
    "data/tweets 2015/tweets_FSF.csv",
    "data/tweets 2015/tweets_INT.csv",
    "data/tweets 2015/tweets_TFP.csv",
    "data/tweets 2015/tweets_TWT.csv"
]

for i,fn in enumerate(filenames_tweets):
    if i == 0:
        df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
    else:
        df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)

C:\Users\valen\AppData\Local\Temp\ipykernel_10732\3352697936.py:19: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
C:\Users\valen\AppData\Local\Temp\ipykernel_10732\3352697936.py:21: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
C:\Users\valen\AppData\Local\Temp\ipykernel_10732\3352697936.py:21: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
C:\Users\valen\AppData\Local\Temp\ipykernel_10732\3352697936.py:21: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
C:\Users\val

In [50]:
df_tweets = df_tweets.dropna(subset = ["user_id"]) # drop those tweet with no userid
df_tweets["user_id"] = df_tweets["user_id"].apply(int) # convert user_id to int
df_tweets['timestamp'] = pd.to_datetime(df_tweets['timestamp']) # convert to timestamp

In [51]:
# Sort based on user_id (ascendign) then tweet time (descending: earliest is first)

df_tweets = df_tweets.sort_values(
    ["user_id", "timestamp"],
    ascending=[True, False]
)

In [52]:
# Create dict of tweets k,v = user_id, listoftweets
# Takes 10-15 mins to run

dict_tweets = {}

for i in range(len(df_tweets)):
    if i % 100000 == 0:
        print(i)
    row = df_tweets.iloc[i,:]
    user_id = row["user_id"]
    text = row["text"]

    if user_id not in dict_tweets.keys():
        dict_tweets[user_id] = [text]
    else:
        dict_tweets[user_id].append(text)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000


In [53]:
#create new column for list of tweets
df_users['tweets_list'] = df_users['id'].map(dict_tweets)

In [54]:
# Save df with tweets corpus
df_users.to_csv("data/combined_twitter_data_with_tweets_corpus.csv")

# Import dataset

In [55]:
# Read df with tweets corpus
df_users = pd.read_csv(base + "/combined_twitter_data_with_tweets_corpus.csv")

In [56]:
df_users.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'name', 'screen_name',
       'statuses_count', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'url', 'lang', 'time_zone',
       'location', 'default_profile', 'default_profile_image', 'geo_enabled',
       'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'protected', 'verified', 'description', 'created_at', 'updated',
       'account_type', 'tweets_list'],
      dtype='object')

## Data Cleaning

### Remove columns that are redundant
Data is redundant in helping us with our problem statement when:
- The data is metadata
- There are too many unique categories

In [57]:
remove_list = ['Unnamed: 0.1', 'Unnamed: 0', 'lang', 'time_zone', 'location', 'profile_banner_url', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https', 'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color', 'profile_link_color', 'utc_offset', 'created_at', 'updated']

In [58]:
df_users = df_users.drop(remove_list, axis = 1)

### Replace NaN values with zeros for binary columns

In [59]:
df_users['default_profile'] = df_users['default_profile'].fillna(0)
df_users['default_profile_image'] = df_users['default_profile_image'].fillna(0)
df_users['geo_enabled'] = df_users['geo_enabled'].fillna(0)
df_users['default_profile_image'] = df_users['default_profile_image'].fillna(0)
df_users['profile_use_background_image'] = df_users['profile_use_background_image'].fillna(0)
df_users['profile_background_tile'] = df_users['profile_background_tile'].fillna(0)

<h2>Train Test Split (85-15)</h2>

In [60]:
#get the target variable - real or fake account type - binary classification problem
df_users = df_users[(df_users['account_type'] == "real") | (df_users['account_type'] == "fake")]
print(df_users['account_type'].value_counts())
df_users['account_type'] = df_users['account_type'].apply(lambda x: 0 if x=="fake" else 1)

train, test = train_test_split(df_users, test_size=0.15, random_state=69, stratify=df_users['account_type'])

fake    8362
real    2751
Name: account_type, dtype: int64


In [61]:
print("train size:", len(train))
print("test size", len(test))

train size: 9446
test size 1667


In [62]:
train['account_type'].value_counts()

0    7108
1    2338
Name: account_type, dtype: int64

<h2>Date Formatting</h2>

In [63]:
# takes around 10 min to run
df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')

In [64]:
df_tweets['created_at_date'] = df_tweets['created_at_formatted'].apply(lambda x: x.date())

<h2>Tweet features</h2>

In [65]:
def tweet_freq(df_users, df_tweets):
    
    # user tweet frequency = total number of tweets / number of user active days 
    # shows how often the user tweets among the days that a user tweets at least once. User activity is defined by whether the user tweets in a given day
    # 1 = user tweets only once per active day 
    # >1 = user tweets more than once a day on average, in the days that the user is active 

    df_tweets_per_day = df_tweets.groupby(by=["user_id"]).agg(tweet_count=('text', 'count'),
                                                              date_count=('created_at_date', lambda x: x.nunique()))

    dict_tweets_average = {user_id: df_tweets_per_day.loc[user_id]['tweet_count'] / df_tweets_per_day.loc[user_id]['date_count'] for user_id in df_tweets_per_day.index}
    #create new column for user tweet frequency 
    df_users['tweet_frequency'] = df_users['id'].map(dict_tweets_average)
    df_users['tweet_frequency'] = df_users['tweet_frequency'].fillna(0)
    return df_users

def tweet_tags_mention(df_users, df_tweets):
    # average number of tags per post = total number of tags used per tweet 
    # average number of mentions per post = total number of mentions per tweet 

    df_tweets['text'] = df_tweets['text'].apply(str) #convert all text to string
    df_tweets['number_of_tags'] = df_tweets['text'].apply(lambda x: x.count("#"))
    df_tweets['number_of_mentions'] = df_tweets['text'].apply(lambda x: x.count("@"))
    tags_dict = df_tweets.groupby(by=["user_id"])['number_of_tags'].sum().to_dict()
    mentions_dict = df_tweets.groupby(by=["user_id"])['number_of_mentions'].sum().to_dict() 

    #create new column for number of tags
    df_users['number_of_tags'] = df_users['id'].map(tags_dict)
    #create new column for number of mentions
    df_users['number_of_mentions'] = df_users['id'].map(mentions_dict)
    
    df_users['number_of_mentions'] = df_users['number_of_mentions'].fillna(0)
    df_users['number_of_tags'] = df_users['number_of_tags'].fillna(0)
    return df_users

In [66]:
# return 0 if weekend, 1 if weekday 
def is_weekday(dt):
    return 0 if dt.weekday() > 4 else 1

# return day of week 
def get_weekday(dt):
    return dt.weekday()

def get_weekend_weekday_frequency(df_tweets, df_users):
    df_tweets['weekday'] = df_tweets['created_at_formatted'].apply(lambda x: is_weekday(x))
    df_tweets_weekday_weekend = df_tweets.groupby(by=["user_id", "weekday"]).agg(tweet_count=('text', 'count'),
                                                          date_count=('created_at_date', lambda x: x.nunique()))
    dict_tweets_weekend = {user_id: df_tweets_weekday_weekend.loc[(user_id, weekday)]['tweet_count'] / df_tweets_weekday_weekend.loc[(user_id, weekday)]['date_count'] for (user_id, weekday) in df_tweets_weekday_weekend.index if weekday == 0}
    df_users['tweet_weekend_frequency'] = df_users['id'].map(dict_tweets_weekend)        
    dict_tweets_weekday = {user_id: df_tweets_weekday_weekend.loc[(user_id, weekday)]['tweet_count'] / df_tweets_weekday_weekend.loc[(user_id, weekday)]['date_count'] for (user_id, weekday) in df_tweets_weekday_weekend.index if weekday == 1}
    df_users['tweet_weekday_frequency'] = df_users['id'].map(dict_tweets_weekday)
    
    df_users['tweet_weekend_frequency'] = df_users['tweet_weekend_frequency'].fillna(0)
    df_users['tweet_weekday_frequency'] = df_users['tweet_weekday_frequency'].fillna(0)
    return df_users      

<h2>Followers To Following Ratio Features

In [67]:
def create_followers_following_ratio(df):
    #followers divide by following (high means popular, low means more following)
    df['following_to_followers_ratio'] = df['friends_count'] / df['followers_count']
    df['following_to_followers_ratio'] = df['following_to_followers_ratio'].fillna(0)
    df['following_to_followers_ratio'] = df['following_to_followers_ratio'].apply(lambda x: 1 if x == np.inf else x)
    return df

<h2>Name Features

In [68]:
def name_features(df):
    #get length of username and screen name
    df['username_length'] = df['name'].apply(lambda x: len(str(x)))
    df['screen_name_length'] = df['screen_name'].apply(lambda x: len(str(x)))
    
    #anything that is not a-z or 0-9 will be blocked, outputs length
    df['username_spec_char_count'] = df['name'].apply(lambda x: len(re.findall(r'[^A-Za-z0-9]+', str(x))))
    df['screen_name_spec_char_count'] = df['screen_name'].apply(lambda x: len(re.findall(r'[^A-Za-z0-9]+', str(x))))
    return df

<h2>Has URL Feature

In [69]:
def has_url_feature(df):
    #1 if has url, 0 if no url
    df['has_url'] = df['url'].apply(lambda x: 0 if pd.isnull(x) else 1)
    return df

<h2>Has Description

In [70]:
def has_description(df):
    #1 if has description, 0 if no description
    df['has_desc'] = df['description'].apply(lambda x: 0 if pd.isnull(x) else 1)
    return df

<h2>Text Features

In [77]:
def clean_texts(df):
    def process_tweets_list(corpus):
        
        corpus_processed = []
        for tweet_list in corpus:
            tweet_list = str(tweet_list)
            row_processed = ""
            
            #replace RT and @
            row_processed = tweet_list.replace("RT", "" ) 
            row_processed = row_processed.replace("@", "" )
            
            row_processed = re.sub(r'http\S+', "", row_processed) #remove any URLs in tweets
            row_processed = re.sub(r'[^\x00-\x7f]', "", row_processed) #remove Non-ASCII characters
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not row_processed == 'nan' else "") # handle NA
            

        return corpus_processed
    
    def process_description(corpus):
        
        corpus_processed = []
        for row in corpus:
            row = str(row)
            # row_processed = re.sub("account", "", row) # remove word "account"
            # row_processed = re.sub("geo-targeted", "", row_processed) # remove phrase "geo-targeted"
            row_processed = re.sub(r'[^\x00-\x7f]', "", row) #remove Non-ASCII characters
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not row_processed == 'nan' else "") # handle NA
            
        return corpus_processed
    
    df["tweets_list_processed"] = process_tweets_list(df["tweets_list"])
    df["description_processed"] = process_description(df["description"])
    
    return df            

In [72]:
class LemmatizeTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, text):
        return [self.lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [73]:
def generate_nlp_features(df):

    tfidf_feature_names = {}

    max_tfidf_features = 100
    
    #tweets
    vect_tweets = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=max_tfidf_features, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    tweets_processed = pd.Series(df["tweets_list_processed"])
    tfidf_fit_tweets = vect_tweets.fit(tweets_processed)
    tweets_tfidf_array = tfidf_fit_tweets.transform(tweets_processed).toarray()
    tweets_tfidf_df = pd.DataFrame(tweets_tfidf_array)
    tweets_tfidf_df.columns = list(map(lambda x: "tweets_" + str(x), tweets_tfidf_df.columns))
    df = pd.concat([df.reset_index(drop=True),tweets_tfidf_df.reset_index(drop=True)], axis=1)
    tfidf_feature_names["tweets"] = list(zip(range(max_tfidf_features),vect_tweets.get_feature_names()))
    
    #description
    vect_description = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=max_tfidf_features, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    description_processed = pd.Series(df["description_processed"])
    tfidf_fit_description = vect_description.fit(description_processed)
    description_tfidf_array = tfidf_fit_description.transform(description_processed).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x: "description_" + str(x), description_tfidf_df.columns))
    df = pd.concat([df.reset_index(drop=True),description_tfidf_df.reset_index(drop=True)], axis=1)
    tfidf_feature_names["description"] = list(zip(range(max_tfidf_features),vect_description.get_feature_names()))


    
    return (df, tfidf_fit_tweets, tfidf_fit_description, tfidf_feature_names)

def nlp_transform_test(df, tfidf_fit_tweets, tfidf_fit_description):
    tweets_tfidf_array = tfidf_fit_tweets.transform(df['tweets_list_processed']).toarray()
    tweets_tfidf_df = pd.DataFrame(tweets_tfidf_array)
    tweets_tfidf_df.columns = list(map(lambda x : "tweets_" + str(x), tweets_tfidf_df.columns))
    df = pd.concat([df.reset_index(drop=True),tweets_tfidf_df.reset_index(drop=True)], axis=1)
    
    description_tfidf_array = tfidf_fit_description.transform(df['description_processed']).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.concat([df.reset_index(drop=True),description_tfidf_df.reset_index(drop=True)], axis=1)
    
    return df

## Has Face Feature (Face Detection with MTCNN)

In [74]:
# import the mapping and attach to df during combine
with open(has_face_pkl_path, 'rb') as f:
    has_face_d = pickle.load(f)

has_face_d = {int(k):v for k,v in has_face_d.items()}

## Graph Analysis: Reciprocity of each user 
- The ratio of the number of relations which are reciprocated over the total number of relations in a directed network

In [75]:
# import the mapping and attach to df during combine
with open(reciprocity_path, 'rb') as f:
    reci_d = pickle.load(f)

<h2>Combine all Feature Generating Functions</h2>

In [78]:
result = tweet_freq(train, df_tweets)
result = tweet_tags_mention(result, df_tweets)
result = get_weekend_weekday_frequency(df_tweets, result)
result = create_followers_following_ratio(result)
result = name_features(result)
result = has_url_feature(result)
result = has_description(result)
result = clean_texts(result)
result, tfidf_fit_tweets, tfidf_fit_description, tfidf_feature_names = generate_nlp_features(result)
result['has_face'] = result['id'].map(has_face_d)
result['reciprocity'] = result['id'].map(reci_d)

c:\Users\valen\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\valen\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(
c:\Users\valen\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\valen\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not No

In [79]:
result

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,...,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity
0,82118682,davide gazzÃ¨,davidegazze,88,19,39,9,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.75
1,1174756808,Carolee Moberly,MoberlycikCarol,19,7,192,0,0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
2,708732794,Julius Kirk,juliuskirkdoq,22,11,241,0,0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
3,2360642101,Magan Skripko,MaganSkripko,47,6,41,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
4,2379608905,Martin Bruley,MartinBruley,34,4,36,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9441,2363037596,Ula Banegas,UlaBanegas,53,6,36,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
9442,1175035327,Jasmine Finkelstein,FinkelsteinupdJ,22,8,194,0,0,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00
9443,67631743,TMJ-Morocco Jobs,tmj_mar_jobs1,49,582,494,0,53,https://t.co/DByWt45HZj,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.00
9444,2370803990,Christy Schnicke,ChristySchnicke,64,11,43,0,0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.00


## Apply same feature engineering on Test data

In [80]:
test = tweet_freq(test, df_tweets)
test = tweet_tags_mention(test, df_tweets)
test = get_weekend_weekday_frequency(df_tweets, test)
test = create_followers_following_ratio(test)
test = name_features(test)
test = has_url_feature(test)
test = has_description(test)
test = clean_texts(test)
test = nlp_transform_test(test, tfidf_fit_tweets, tfidf_fit_description)
test['has_face'] = test['id'].map(has_face_d)
test['reciprocity'] = test['id'].map(reci_d)

In [81]:
print(len(test.columns))
print(len(result.columns))

for i in result.columns:
    if i not in test.columns:
        print(i)

236
236


## Export to CSV

In [82]:
result.to_csv("data/twitter_data_train.csv", index=False)
test.to_csv("data/twitter_data_test.csv", index=False)

In [83]:
result[result.id==1]

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,...,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity


In [84]:
test[test.id==1]

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,...,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity


In [85]:
import pickle
tfidf_feature_names

with open('data/tfidf_feature_names.pkl', 'wb') as f:
    pickle.dump(tfidf_feature_names, f)

In [86]:
tfidf_feature_names

{'tweets': [(0, 'al'),
  (1, 'always'),
  (2, 'amp'),
  (3, 'anche'),
  (4, 'back'),
  (5, 'best'),
  (6, 'cant'),
  (7, 'che'),
  (8, 'check'),
  (9, 'chi'),
  (10, 'ci'),
  (11, 'come'),
  (12, 'con'),
  (13, 'cool'),
  (14, 'da'),
  (15, 'day'),
  (16, 'de'),
  (17, 'dei'),
  (18, 'del'),
  (19, 'della'),
  (20, 'di'),
  (21, 'dont'),
  (22, 'e'),
  (23, 'essere'),
  (24, 'get'),
  (25, 'gli'),
  (26, 'go'),
  (27, 'going'),
  (28, 'good'),
  (29, 'got'),
  (30, 'great'),
  (31, 'great introduction'),
  (32, 'ha'),
  (33, 'history'),
  (34, 'ho'),
  (35, 'il'),
  (36, 'im'),
  (37, 'introduction'),
  (38, 'io'),
  (39, 'know'),
  (40, 'la'),
  (41, 'le'),
  (42, 'learn'),
  (43, 'life'),
  (44, 'like'),
  (45, 'lo'),
  (46, 'lol'),
  (47, 'look'),
  (48, 'love'),
  (49, 'mai'),
  (50, 'make'),
  (51, 'man'),
  (52, 'mi'),
  (53, 'money'),
  (54, 'much'),
  (55, 'na'),
  (56, 'need'),
  (57, 'never'),
  (58, 'new'),
  (59, 'non'),
  (60, 'one'),
  (61, 'people'),
  (62, 'per'),
  (63

## Get multiclass label

In [87]:
train_df = pd.read_csv("data/twitter_data_train.csv")
test_df = pd.read_csv("data/twitter_data_test.csv")

In [88]:
train_df[train_df.id==1]

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,...,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity


In [89]:
df_import = pd.read_csv("data/cresci2017/traditional_spambots_4.csv")
#df_import = pd.read_csv("/Users/ivankoh/Downloads/traditional_spambots_4.csv")

In [90]:
df_import


,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,lang,...,protected,verified,notifications,description,contributors_enabled,following,created_at,timestamp,crawled_at,updated
0,21478911,TMJ- CLT Util Jobs,tmj_clt_util,4,344,295,0,5,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:04:47 +0000 2009,2009-02-21 13:04:47,2016-03-15 13:48:59,2016-03-15 13:48:59
1,21479094,TMJ - SFO Util Jobs,tmj_sfo_util,3,353,322,0,16,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:09:26 +0000 2009,2009-02-21 13:09:26,2016-03-15 13:48:59,2016-03-15 13:48:59
2,21479204,TMJ - WAS Util Jobs,tmj_dc_util,1,323,294,0,2,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:12:01 +0000 2009,2009-02-21 13:12:01,2016-03-15 13:48:59,2016-03-15 13:48:59
3,21479275,TMJ - JAX Util Jobs,tmj_jax_util,4,311,292,0,4,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:13:37 +0000 2009,2009-02-21 13:13:37,2016-03-15 13:48:59,2016-03-15 13:48:59
4,21479334,TMJ - CHI Util Jobs,tmj_chi_util,6,339,298,0,7,https://t.co/DByWt45HZj,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Utilities...,NaN,NaN,Sat Feb 21 12:15:21 +0000 2009,2009-02-21 13:15:21,2016-03-15 13:48:59,2016-03-15 13:48:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,818079408,Columbia Facil. Mgmt,tmj_cae_facmgmt,41,169,151,0,12,http://t.co/TKb82K4tFJ,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Facilitie...,NaN,NaN,Tue Sep 11 20:00:43 +0000 2012,2012-09-11 22:00:43,2016-03-15 13:49:15,2016-03-15 13:49:15
1124,846108068,NJ Business/Mgmt,tmj_nj_mgmt,218,332,245,0,31,https://t.co/DByWt4njnT,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Business/...,NaN,NaN,Tue Sep 25 19:35:53 +0000 2012,2012-09-25 21:35:53,2016-03-15 13:49:15,2016-03-15 13:49:15
1125,857076271,PR Customer Ser.,tmj_ptr_cstsrv,17,530,300,0,8,http://t.co/paWgrkSWfT,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Customer ...,NaN,NaN,Mon Oct 01 21:47:23 +0000 2012,2012-10-01 23:47:23,2016-03-15 13:49:15,2016-03-15 13:49:15
1126,872126906,PR Soft Dev Java,tmj_ptr_itjava,2,247,202,0,1,http://t.co/OnFSpFHC7G,en,...,NaN,NaN,NaN,Follow this account for geo-targeted Software ...,NaN,NaN,Wed Oct 10 16:44:09 +0000 2012,2012-10-10 18:44:09,2016-03-15 13:49:15,2016-03-15 13:49:15


In [91]:


ff = pd.read_csv("data/cresci2017/fake_followers.csv")
ga = pd.read_csv("data/cresci2017/genuine_accounts.csv") # real
ss1 = pd.read_csv("data/cresci2017/social_spambots_1.csv")
ss2 = pd.read_csv("data/cresci2017/social_spambots_2.csv")
ss3 = pd.read_csv("data/cresci2017/social_spambots_3.csv")
ts1 = pd.read_csv("data/cresci2017/traditional_spambots_1.csv")
ts2 = pd.read_csv("data/cresci2017/traditional_spambots_2.csv")
ts3 = pd.read_csv("data/cresci2017/traditional_spambots_3.csv")
ts4 = pd.read_csv("data/cresci2017/traditional_spambots_4.csv")


ga = pd.concat([ga, pd.read_csv("data/cresci2015/E13.csv")], axis=0)  # real
ff = pd.concat([ff, pd.read_csv("data/cresci2015/FSF.csv")], axis=0) 
ff = pd.concat([ff, pd.read_csv("data/cresci2015/INT.csv")], axis=0) 
ga = pd.concat([ga, pd.read_csv("data/cresci2015/TFP.csv")], axis=0)  # real
ff = pd.concat([ff, pd.read_csv("data/cresci2015/TWT.csv")], axis=0) 

REAL_ACCOUNT = 0
FAKE_FOLLOWER = 1
POLITICAL_SPAMMERS = 2
ADVERTISING_SPAMMERS = 3
OTHER_SPAMMERS = 4
SCAMMERS = 5

def gettype(id):
    
    if id in list(ff.id):
        return FAKE_FOLLOWER

    elif id in list(ga.id):
        return REAL_ACCOUNT

    elif id in list(ss1.id):
        return POLITICAL_SPAMMERS
    elif id in list(ss2.id):
        return ADVERTISING_SPAMMERS
    elif id in list(ss3.id):
        return ADVERTISING_SPAMMERS

    elif id in list(ts1.id):
        return OTHER_SPAMMERS
    elif id in list(ts2.id):
        return SCAMMERS
    elif id in list(ts3.id):
        return ADVERTISING_SPAMMERS
    elif id in list(ts4.id):
        return ADVERTISING_SPAMMERS

    else:
        print(id)
        return -1
        


In [92]:
gettype(1)

1


-1

In [93]:
train_df["account_type_multi"] = train_df["id"].apply(lambda id: gettype(id))
test_df["account_type_multi"] = test_df["id"].apply(lambda id: gettype(id))

In [94]:
train_df["account_type_multi"].unique()


array([0, 1, 3, 2, 5, 4], dtype=int64)

In [95]:
test_df["account_type_multi"].unique()

array([1, 0, 3, 4, 2, 5], dtype=int64)

In [96]:
train_df.to_csv("data/twitter_data_train_multiclass.csv")
test_df.to_csv("data/twitter_data_test_multiclass.csv")